# Creating and Valuing a CDS Contract

Replicating Markit Pricing on 20 Aug 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.200 - This build:  14 Jul 2021 at 13:18 #
# **** NEW PEP8 COMPLIANT VERSION -- PLEASE UPDATE YOUR CODE  **** #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Creating a CDS Contract

In [3]:
valuation_date = Date(20, 8, 2020)
effective_date = Date(21, 8, 2020)
maturity_date = Date(20, 6, 2025)
running_coupon = 0.05
notional = ONE_MILLION
long_protection = True

In [5]:
cds_contract = CDS(effective_date, maturity_date, running_coupon, notional, long_protection)

## Build Ibor Curve

In [6]:
settlement_date = Date(24, 8, 2020)

In [8]:
dcType = DayCountTypes.ACT_360
depo1 = IborDeposit(settlement_date, "1M", 0.001709, dcType)
depo2 = IborDeposit(settlement_date, "2M", 0.002123, dcType)
depo3 = IborDeposit(settlement_date, "3M", 0.002469, dcType)
depo4 = IborDeposit(settlement_date, "6M", 0.003045, dcType)
depo5 = IborDeposit(settlement_date, "12M", 0.004449, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

swapType = SwapTypes.PAY
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swap1 = IborSwap(settlement_date,"2Y", swapType, 0.002155,fixedFreq,dcType)
swap2 = IborSwap(settlement_date,"3Y", swapType, 0.002305,fixedFreq,dcType)
swap3 = IborSwap(settlement_date,"4Y", swapType, 0.002665,fixedFreq,dcType)
swap4 = IborSwap(settlement_date,"5Y", swapType, 0.003290,fixedFreq,dcType)
swap5 = IborSwap(settlement_date,"6Y", swapType, 0.004025,fixedFreq,dcType)
swap6 = IborSwap(settlement_date,"7Y", swapType, 0.004725,fixedFreq,dcType)
swap7 = IborSwap(settlement_date,"8Y", swapType, 0.005430,fixedFreq,dcType)
swap8 = IborSwap(settlement_date,"9Y", swapType, 0.006075,fixedFreq,dcType)
swap9 = IborSwap(settlement_date,"10Y", swapType, 0.006640,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9]

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

# Build a CDS Curve

In [10]:
cdsSpread = 0.01

In [11]:
cds1 = CDS(settlement_date, "6M", cdsSpread)
cds2 = CDS(settlement_date, "1Y", cdsSpread)
cds3 = CDS(settlement_date, "2Y", cdsSpread)
cds4 = CDS(settlement_date, "3Y", cdsSpread)
cds5 = CDS(settlement_date, "4Y", cdsSpread)
cds6 = CDS(settlement_date, "5Y", cdsSpread)
cds7 = CDS(settlement_date, "7Y", cdsSpread)
cds8 = CDS(settlement_date, "10Y", cdsSpread)

In [12]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [13]:
recovery_rate = 0.40

In [15]:
issuer_curve = CDSCurve(valuation_date, cdss, libor_curve, recovery_rate)

In [16]:
print(issuer_curve)

OBJECT TYPE: CDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 0.5808219,  0.9901921
 1.0849315,  0.9817980
 2.0849315,  0.9986816
 3.0849315,  0.9804199
 4.0876712,  0.9617070
 5.0876712,  0.9461404
 7.0876712,  0.9130451
10.0904110,  0.8664844


In [17]:
cds_contract.print_flows(issuer_curve)

PAYMENT_DATE      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
    21-SEP-2020   0.252778     12638.89     0.999848     0.998513     12618.18
    21-DEC-2020   0.252778     12638.89     0.999079     0.994298     12555.25
    22-MAR-2021   0.252778     12638.89     0.998005     0.990100     12488.80
    21-JUN-2021   0.252778     12638.89     0.996528     0.985940     12417.93
    20-SEP-2021   0.252778     12638.89     0.995505     0.981798     12353.06
    20-DEC-2021   0.252778     12638.89     0.995554     0.985980     12406.29
    21-MAR-2022   0.252778     12638.89     0.995603     0.990181     12459.76
    20-JUN-2022   0.252778     12638.89     0.995652     0.994399     12513.45
    20-SEP-2022   0.255556     12777.78     0.995495     0.998682     12703.45
    20-DEC-2022   0.252778     12638.89     0.994849     0.994097     12499.57
    20-MAR-2023   0.250000     12500.00     0.994211     0.989584     12298.18
    20-JUN-2023   0.255556     12777.78     0.993558   

# Valuation

In [1]:
spd = cds_contract.par_spread(settlement_date, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

NameError: name 'cds_contract' is not defined

In [19]:
cds_contract.value_fast_approx(settlement_date, 0.004, 0.01, 0.40, 0.40)

(-195533.6454106767, -187200.31207734335, 533.6950180429558, 44.32197353540687)

In [20]:
v = cds_contract.value(settlement_date, issuer_curve, recovery_rate)

In [21]:
dirty_pv = v['dirty_pv'] 
clean_pv = v['clean_pv']

In [22]:
print("DIRTY VALUE  %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

DIRTY VALUE    -199370.92
CLEAN VALUE   -191037.58


In [23]:
cleanp = cds_contract.clean_price(settlement_date, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   119.039676


In [24]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 60.0


In [25]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON -8333.333333333334


In [26]:
prot_pv = cds_contract.protection_leg_pv(settlement_date, issuer_curve, recovery_rate)
print("PROTECTION_PV", prot_pv)

PROTECTION_PV 47813.122593011336


In [27]:
premPV = cds_contract.premium_leg_pv(settlement_date, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 247184.03808119084


In [28]:
cds_contract.risky_pv01(settlement_date, issuer_curve)

{'dirty_rpv01': 4.943680761623817, 'clean_rpv01': 4.77701409495715}

## Risk Measures

In [29]:
cds_contract.credit_dv01(settlement_date, issuer_curve, recovery_rate)

514.3827831871749

In [30]:
cds_contract.interest_dv01(settlement_date, issuer_curve, recovery_rate)

4.157797746738652

Copyright (c) 2020 Dominic O'Kane